# Introduction to Recommender Systems

<p align="center">
    <img width="721" alt="cover-image" src="https://user-images.githubusercontent.com/49638680/204351915-373011d3-75ac-4e21-a6df-99cd1c552f2c.png">
</p>

---

## Non-Personalised Recommendations

Let's introduce a non-personalised recommendation system. Before proceeding let's point out the reasons for the non-personalised recommendations:

1. New users: we know little about them (_cold-start_ problem).
2. Simple implementation but beneficial.
3. Communities share the same behaviour, hence same recommendations.
4. Applications to cases where personalised recommendations are impossible.

As an example, think about the weekly review of books or restaurants on newspaper articles.
These are also called __stereotyped recommender system__.

## A code implementation of a non-personalised recommendation system

The aim of this notebook is to provide an implementation of a non-personalised recommendation system.


In [1]:
# Import libraries
import numpy as np
import pandas as pd
from typing import List

from utils import non_pers_rec as npr
from tests.non_pers_tests import test_get_recent_liked_movie, test_get_movie_genre

import matplotlib.pyplot as plt

# set plot size
plt.rcParams["figure.figsize"] = (20, 13)
%matplotlib inline
%config InlineBackend.figure_format = "retina"

We are going to use only numpy to build the recommender for users that have not rated any movie, by suggesting the most popular movie in the catalogue.

### Import data

The data comes from the famous [movielens dataset](https://grouplens.org/datasets/movielens/), here we import the data into dataframes in order to make a nice data analysis before digging into the model.

In [2]:
df_rating = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u1.base",
    sep="\t",
    engine="python",
    header=None,
)
df_rating.columns = ["UserId", "MovieId", "Rating", "Timestamp"]
df_rating_test = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u1.test",
    sep="\t",
    engine="python",
    header=None,
)

df_users = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.user",
    sep="|",
    engine="python",
    header=None,
)
df_users.columns = ["UserId", "Age", "Gender", "Occupation", "ZipCode"]
df_users.set_index("UserId", inplace=True)
df_items = pd.read_csv(
    "http://files.grouplens.org/datasets/movielens/ml-100k/u.item",
    sep="|",
    engine="python",
    encoding="ISO-8859-1",
    header=None,
)
df_items.columns = [
    "MovieId",
    "Title",
    "Date",
    "VideoReleaseDate",
    "Url",
    "unknown",
    "Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western",
]
df_items.set_index("MovieId", inplace=True)

In [3]:
df_rating

,UserId,MovieId,Rating,Timestamp
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In order to compose the training data as we have seen in lectures, we need to create a matrix having users in rows and movies in columns (or its equivalent transpose matrix).

This can be easily done by a pivot operator in pandas.

In [4]:
df_matrix = df_rating.pivot(index="UserId", columns="MovieId", values="Rating")

n_users = len(df_users)
n_items = len(df_items)

In [5]:
df_matrix

MovieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
UserId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,NaN,4.0,1.0,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
940,NaN,NaN,NaN,2.0,NaN,NaN,4.0,5.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
941,5.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As one can see, the matrix is full of nan values.

### Calculate the most popular movie

Let's look at the movie colums. We can create a vector, summing over the colums, to get a _total score_ measuring how much that movie has been rated.

Thanks to pandas methods, this can be done by one line of code.

In [6]:
movie_scores = df_matrix.sum(axis=0, skipna=True)
movie_scores

MovieId
1       1491.0
2        334.0
3        225.0
4        603.0
5        228.0
         ...  
1678       1.0
1679       3.0
1680       2.0
1681       3.0
1682       3.0
Length: 1650, dtype: float64

Now we need to store the movie score in the corresponding dataframe.
Since the movie score is a pandas series, whose index is the same as the index of the movie dataframe, we can simply add a column as follows.

In [7]:
df_items["score"] = movie_scores
df_items

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,1,0,0,334.0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,225.0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,603.0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,228.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,3.0
1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2.0


### Giving recommendations

Hence, we are ready to give recommendations. 

We split the problem into four steps of growing difficulty.

1. We always recommend the most popular movie.
2. We recommend the most popular movie in the same genre of the best rated movie per each user.
3. We look at the last rated movie (whose rating is over 4) for each user and we recommend the most popular movie in the same genre.
4. We do as in $3.$ but with a random distribution, we pick uniformly amongst the first $10$ most popular movies.

#### Most popular movie recommendation

The first point is the easiest one, we only need to look at the movie whose score is the highest one.

In [8]:
df_items.iloc[[df_items.score.argmax()]]

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0


Hence, let's write a recommendation function.

In [9]:
def recommend_one():
    """Function to return the recommendations based on the most popular criterion."""
    return df_items.loc[[df_items.score.argmax() + 1]]

In [10]:
recommend_one()

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0


This function returns just the most popular movie. However, we may want to consider multiple recommendations: hence we want to return the first then movies with the highest popularity score.

In [11]:
def recommend_n(n_recommendations: int = 10) -> pd.DataFrame:
    """Function to return recommendations based on the most popular criterion.
    Parameters
    ----------
    n_recommendations: int
        Number of recommendations to return.
        default: 10

    Returns
    -------
    pd.DataFrame
        The dataframe containing the recommendations in order of preference.
    """
    return df_items.sort_values(by="score", ascending=False).iloc[:n_recommendations]

In [12]:
recommend_n()

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1693.0
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1623.0
258,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1537.0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
174,Raiders of the Lost Ark (1981),01-Jan-1981,NaN,http://us.imdb.com/M/title-exact?Raiders%20of%...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1461.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1413.0
288,Scream (1996),20-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Scream%20(1996),0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1347.0


#### Most popular movie in the same genre recommendation

Now we start to include the user history in some sense. This is not personalised, as we never use user features, but we focus on the last movie the user liked in order to make a recommendation about a new movie.

This is a first example of _content based recommendations_.

First, we need to see which movie the user liked most recently. This can be done by looking at the "timestamp" column in rating table.

<details>
    <summary><b>HINT</b></summary> 
    
    You can find the implemented function in the utils module `non_pers_rec.py`. 
    Try to implement the function on your own before looking at the given implementation.
</details>

In [13]:
def get_recent_liked_movie(df_rating: pd.DataFrame, user_id: int, threshold: float = 4.0) -> int:
    """Function to get the id of the movie that the user has liked most recently.

    Parameters
    ----------
    df_rating : pd.DataFrame
        The data frame containing the movie ratings per user.
    user_id : int
        The id of the user we are interested in.
    threshold : float
        The rating threshold, to determine whether a user appreciated the movie.
        default: 4.0

    Returns
    -------
    int
        The movie id that the user has liked most recently.
    """
    # Complete the function

In [13]:
### Unit tests ###
test_get_recent_liked_movie(get_recent_liked_movie)

 All tests passed.


##### Extract the genre information

The idea is to use the just-found id, to extract the genre information out of the `df_items` dataframe, and then recommend the most popular movies in the same genre.

**Note**: The `df_items` dataframe is multi-hot encoding the genre information, hence a movie can have multiple genres and they are stored as ones in the corresponding columns of the dataframe.

To fix this issue, we define a pandas series populated converting the multi-hot encoding columns into a single column.

In [13]:
genre_cols = ["Action",
    "Adventure",
    "Animation",
    "Children",
    "Comedy",
    "Crime",
    "Documentary",
    "Drama",
    "Fantasy",
    "Film-Noir",
    "Horror",
    "Musical",
    "Mystery",
    "Romance",
    "Sci-Fi",
    "Thriller",
    "War",
    "Western"] 


genre_series = df_items[genre_cols].apply(lambda row: row[row == 1].index.tolist(), axis=1)
genre_series

MovieId
1       [Animation, Children, Comedy]
2       [Action, Adventure, Thriller]
3                          [Thriller]
4             [Action, Comedy, Drama]
5            [Crime, Drama, Thriller]
                    ...              
1678                          [Drama]
1679              [Romance, Thriller]
1680                 [Drama, Romance]
1681                         [Comedy]
1682                          [Drama]
Length: 1682, dtype: object

Once the list of genres has been used to create the corresponding column, we can extract such information passing the movie id.

In [14]:
def get_genre(movie_id: int) -> List[str]:
    """Function to get the genre of the inserted movie.

    Parameters
    ----------
    movie_id : int
        The id of the movie whose genre is to be returned.

    Returns
    -------
    list of str
        the genres of the inserted movie.
    """
    
    # Complete the function

In [15]:
### Unit tests ###
test_get_movie_genre(get_genre)

 All tests passed.


At this stage, we need to make a choice. 
In particular, we have to decide how to handle the case where one movie is classified in more than one genre.

Our choice is the following:
> If a movie has more than one genre, then we take the most popular movies in all the genres appearing in the list.

Let's write a code to recommend the first $n$ movies in the genre of the last positively evaluated movie.

In [23]:
def recommend_genre_n(n_recommendations: int = 10, genres: List[str] = None) -> pd.DataFrame:
    """Function to return recommendations based on the most popular criterion in a given genre list.
    Parameters
    ----------
    n_recommendations: int
        Number of recommendations to return.
        default: 10
    genres: List of strings
        The list of genres in which to return the recommendations.
        If not provided, the function will return the recommendations based on all genres.
        default: None
        
    Returns
    -------
    pd.DataFrame
        The dataframe containing the recommendations in order of preference.
    """

    if genres is not None:
        # Filter movies by genre
        mask = df_items[genres].eq(1).all(axis=1)
        return df_items[mask].sort_values(by="score", ascending=False).iloc[:n_recommendations]
    else:
        return recommend_n(n_recommendations=n_recommendations)

Let's make sure that the function works as expected.

In [25]:
# Call on all the genres.
recommend_genre_n()

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1693.0
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1623.0
258,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1537.0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
174,Raiders of the Lost Ark (1981),01-Jan-1981,NaN,http://us.imdb.com/M/title-exact?Raiders%20of%...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1461.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1413.0
288,Scream (1996),20-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Scream%20(1996),0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1347.0


In [26]:
genres = ["Crime", "Drama"]

recommend_genre_n(genres=genres)

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1623.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0
56,Pulp Fiction (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Pulp%20Fictio...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1252.0
187,"Godfather: Part II, The (1974)",01-Jan-1974,NaN,http://us.imdb.com/M/title-exact?Godfather:%20...,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,736.0
182,GoodFellas (1990),01-Jan-1990,NaN,http://us.imdb.com/M/title-exact?GoodFellas%20...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,698.0
403,Batman (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Batman%20(1989),0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,553.0
293,Donnie Brasco (1997),28-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Donnie%20Bras...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,494.0
327,Cop Land (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Cop+Land+(1997),0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,469.0
628,Sleepers (1996),18-Oct-1996,NaN,http://us.imdb.com/M/title-exact?Sleepers%20(1...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,466.0


**Excercise** 

What is missing at this stage is a function combining the work we have done above such that given a user id, we can have recommendations based on what the user liked more recently. 
_Writing such a function is left as an exercise._

Be sure to handle the case of a new user, _e.g._ a user_id that is not in the database.
There is no need to write a lot of code, try to use the functions we defined above.

Here is an example docstring written for you.

```python
def get_recommendations(user_id: int) -> pd.DataFrame:
    """Function to get recommendations for a user identified by its user_id.
    These are in the same genres as the most recently liked movie.

    Parameters
    ----------
    user_id : int
        The id of the user for whom recommendations are generated.
    
    Returns
    -------
    pd.DataFrame
        The recommendations for the given user, in the form of a pd.DataFrame.
    """
```

---

#### Picking recommendations from a distribution

Finally, we can use a strategy to randomly pick recommendations out of a distribution.

Indeed, we do not want to recommend movies always in order of score, but sometimes, it is more efficient to pick randomly out of a bucket of movies.

To do so, we only need to modify the function above as follows.


In [46]:
def recommend_genre_n(n_recommendations: int = 10, genres: List[str] = None, temp: float = 1.0) -> pd.DataFrame:
    """Function to return recommendations based on the most popular criterion in a given genre list.
    Parameters
    ----------
    n_recommendations: int
        Number of recommendations to return.
        default: 10
    genres: List of strings
        The list of genres in which to return the recommendations.
        If not provided, the function will return the recommendations based on all genres.
        default: None
    temp: float
        Parameter setting the randomness of the recommendations. The higher is the temperature, more random is the recommendation.
        default: 1.0
        
    Returns
    -------
    pd.DataFrame
        The dataframe containing the recommendations in order of preference.
    """

    if genres is not None:
        # Filter movies by genre
        mask = df_items[genres].eq(1).all(axis=1)
        df_recommend = df_items[mask].sort_values(by="score", ascending=False).iloc[:n_recommendations]
    else:
        df_recommend = recommend_n(n_recommendations=n_recommendations)

    # Define and normalise vector of probabilities
    probs = [np.exp(-i/temp) for i in range(len(df_recommend))]
    probs = np.array(probs)/np.sum(probs)
    

    rnd_indices = np.random.choice(df_recommend.index, n_recommendations, p=probs, replace=False)
    return df_recommend.loc[rnd_indices]

In [52]:
# Let's verify that the recommendations order changes with temperature.
display(recommend_genre_n(temp=10))
display(recommend_genre_n(temp=2))
display(recommend_genre_n(temp=200))

,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1693.0
300,Air Force One (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Air+Force+One...,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1287.0
288,Scream (1996),20-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Scream%20(1996),0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1347.0
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1623.0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1413.0
258,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1537.0
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0


,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1693.0
100,Fargo (1996),14-Feb-1997,NaN,http://us.imdb.com/M/title-exact?Fargo%20(1996),0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1623.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0
258,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1537.0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
174,Raiders of the Lost Ark (1981),01-Jan-1981,NaN,http://us.imdb.com/M/title-exact?Raiders%20of%...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1461.0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1413.0
300,Air Force One (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Air+Force+One...,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1287.0


,Title,Date,VideoReleaseDate,Url,unknown,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,score
MovieId,,,,,,,,,,,,,,,,,,,,,
300,Air Force One (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Air+Force+One...,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1287.0
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,1491.0
286,"English Patient, The (1996)",15-Nov-1996,NaN,http://us.imdb.com/M/title-exact?English%20Pat...,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,1413.0
127,"Godfather, The (1972)",01-Jan-1972,NaN,"http://us.imdb.com/M/title-exact?Godfather,%20...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1448.0
174,Raiders of the Lost Ark (1981),01-Jan-1981,NaN,http://us.imdb.com/M/title-exact?Raiders%20of%...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1461.0
50,Star Wars (1977),01-Jan-1977,NaN,http://us.imdb.com/M/title-exact?Star%20Wars%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,2110.0
181,Return of the Jedi (1983),14-Mar-1997,NaN,http://us.imdb.com/M/title-exact?Return%20of%2...,0,1,1,0,0,0,...,0,0,0,0,1,1,0,1,0,1693.0
288,Scream (1996),20-Dec-1996,NaN,http://us.imdb.com/M/title-exact?Scream%20(1996),0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1347.0
258,Contact (1997),11-Jul-1997,NaN,http://us.imdb.com/Title?Contact+(1997/I),0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1537.0


#### Further exercises

Try to write a test function that outputs the recommendations for an user that recently liked "the Godfather part 1".

* Verify that the other two movies of the Godfather saga are present in the recommendations.
* Slightly modify the code above in order to exclude a movie the user has already seen and evaluated.
